In [2]:
pip install --force-reinstall torch==1.12.1+cu113 --extra-index-url https://download.pytorch.org/whl/

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/
  Using cached https://download.pytorch.org/whl/cu113/torch-1.12.1%2Bcu113-cp38-cp38-linux_x86_64.whl (1837.7 MB)
  Using cached typing_extensions-4.7.1-py3-none-any.whl (33 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Rolling back uninstall of torch
  Moving to /home/pkatlan/miniconda3/envs/recommender/bin/convert-caffe2-to-onnx
   from /tmp/pip-uninstall-wnajy5v4/convert-caffe2-to-onnx
  Moving to /home/pkatlan/miniconda3/envs/recommender/bin/convert-onnx-to-caffe2
   from /tmp/pip-uninstall-wnajy5v4/convert-onnx-to-caffe2
  Moving to /home/pkatlan/miniconda3/envs/recommender/bin/torchr

In [3]:
from summac.model_summac import SummaCZS, SummaCConv

model_zs = SummaCZS(granularity="sentence", model_name="vitc", device="cuda") # If you have a GPU: switch to: device="cuda"
model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="sentence", nli_labels="e", device="cuda", start_file="default", agg="mean")


<All keys matched successfully>


In [4]:
document = """Scientists are studying Mars to learn about the Red Planet and find landing sites for future missions.
One possible site, known as Arcadia Planitia, is covered instrange sinuous features.
The shapes could be signs that the area is actually made of glaciers, which are large masses of slow-moving ice.
Arcadia Planitia is in Mars' northern lowlands."""

summary1 = "There are strange shape patterns on Arcadia Planitia. The shapes could indicate the area might be made of glaciers. This makes Arcadia Planitia ideal for future missions."
score_zs1 = model_zs.score([document], [summary1])
score_conv1 = model_conv.score([document], [summary1])
print("[Summary 1] SummaCZS Score: %.3f; SummacConv score: %.3f" % (score_zs1["scores"][0], score_conv1["scores"][0])) # [Summary 1] SummaCZS Score: 0.582; SummacConv score: 0.536

summary2 = "There are strange shape patterns on Arcadia Planitia. The shapes could indicate the area might be made of glaciers."
score_zs2 = model_zs.score([document], [summary2])
score_conv2 = model_conv.score([document], [summary2])
print("[Summary 2] SummaCZS Score: %.3f; SummacConv score: %.3f" % (score_zs2["scores"][0], score_conv2["scores"][0])) # [Summary 2] SummaCZS Score: 0.877; SummacConv score: 0.709

[Summary 1] SummaCZS Score: 0.583; SummacConv score: 0.536
[Summary 2] SummaCZS Score: 0.877; SummacConv score: 0.709


/home/pkatlan/miniconda3/envs/recommender/lib/python3.8/site-packages/summac/model_summac.py:267: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  histograms = torch.FloatTensor(histograms).to(self.device)


In [5]:
import json
with open("dataset.json", "r") as f:
    content = f.read()
dataset = json.loads(content)

In [6]:
import pandas as pd
df = pd.DataFrame(dataset)
df["wiki_bio_text"][0]

'Sir John Russell Reynolds, 1st Baronet (22 May 1828 – 29 May 1896) was a British neurologist and physician. Reynolds was born in Romsey, Hampshire, as the son of John Reynolds, an independent minister, and the grandson of Dr. Henry Revell Reynolds. He received general education from his father, and was educated in his profession at University College, London, where he obtained three gold medals in the medical school. In 1851, he graduated M.B. in the University of London, and obtained a scholarship and gold medal in medicine. In 1852, he took the degree of M.D., and began practice in Leeds. He soon after moved to London, and took a house, 38 Grosvenor Street, in which Dr. Marshall Hall had lived. Hall announced to his patients in a printed circular that Reynolds had succeeded him in practice. Such procedure was contrary to a recognised understanding among physicians, and Hall incurred the censure of the College of Physicians. Reynolds, who was ignorant of Hall\'s intention, was in no 

In [11]:
%%time
document = df['wiki_bio_text'][0]
summary = df['gpt3_text'][0]
scores = {}
scores['SummaCZS'] = model_zs.score([document], [summary])['scores'][0]
#scores['SummaCZS_no_con'] =model_zs_nocon.score([document], [summary])['scores'][0]
scores['SummaConv'] = model_conv.score([document], [summary])['scores'][0]

CPU times: user 2.97 ms, sys: 0 ns, total: 2.97 ms
Wall time: 2.4 ms


In [8]:
scores

{'SummaCZS': -0.5527191162109375, 'SummaConv': 0.24911682307720184}

In [15]:
zs = []
conv = []
for i in range(len(dataset)):
    document = df['wiki_bio_text'][i]
    summary = df['gpt3_text'][i]

        #summary1 = "There are strange shape patterns on Arcadia Planitia. The shapes could indicate the area might be made of glaciers. This makes Arcadia Planitia ideal for future missions."
    score_zs1 = model_zs.score([document], [summary])
    score_conv1 = model_conv.score([document], [summary])
    #print(score_zs1["scores"][0])
    zs.append(score_zs1["scores"][0])
    conv.append(score_conv1["scores"][0])
    #print("[Summary] SummaCZS Score: %.3f; SummacConv score: %.3f" % (score_zs1["scores"][0], score_conv1["scores"][0])) # [Summary 1] SummaCZS Score: 0.582; SummacConv score: 0.536

summac_scores = pd.DataFrame(
    {
        'zs': zs,
        'conv': conv
        #'no_con': no_con
    })     
    
    

In [16]:
summac_scores

,zs,conv
0,-0.552719,0.249117
1,-0.440639,0.209684
2,-0.551670,0.223239
3,-0.709837,0.215711
4,-0.777010,0.218822
...,...,...
233,-0.904063,0.231275
234,-0.052294,0.234655
235,-0.553773,0.217451
236,-0.508882,0.230450


In [17]:
summac_scores.to_csv('summac_scores.csv')